Juntando todas as bases em um banco de dados 

Esse script tem por finalidade unir as bases de dados de todos os estados que já foram corrigidas. Assim tonarndo uma unica base, adicionando as colunas: ano, sexo, tipologia e estado. Dessa forma podendo realizar analise descritiva dos obitos por varios niveis.

### Verificando diretorio 

In [1]:
import os
import pandas as pd

os.getcwd() # verifica o diretorio

'C:\\Users\\Administrador\\Documents\\Mortalidade_no_semiarido\\PIBIC\\_SCHENIA_2023_2024\\4_analise da base'

------------------------------------------------------------

### Função I

Essa função abre o arquivo em excel com os dados redistribuidos e corrigidos. 
* faz o tratamento selecionando apenas os grande grupos de causas: selecionando apenas as linhas em que todas as letras da coluna Causa - CID-BR-10 são maiusculas. 
* adiciona as colunas tipologia, sexo e ano, com base no nome de cada aba.
* Cada aba gera um df em que adiciona as colunas.
* apos isso concaneta todos os df em um unico df (cada dataframe é correspondente a uma aba do excel)

In [2]:
import pandas as pd

def listar_abas(caminho_arquivo):
    # Usa a função pd.ExcelFile para ler o arquivo Excel
    excel_file = pd.ExcelFile(caminho_arquivo)
    
    # Obtém a lista de nomes das abas
    nomes_abas = excel_file.sheet_names
    
    return nomes_abas

def processar_dataframe(caminho_arquivo, nome_aba):
    # Lê a aba específica do arquivo Excel
    df = pd.read_excel(caminho_arquivo, sheet_name=nome_aba)
    
    # Mantém apenas as 20 primeiras colunas
    df = df.iloc[:, :20]
    
    # Encontra a linha que contém "Total" na coluna 'Causa - CID-BR-10'
    linha_chave = df[df['Causa - CID-BR-10'].str.contains('Total', na=False)].index
    
    if not linha_chave.empty:
        # Pega o índice da linha chave
        indice_linha_chave = linha_chave[0]
        
        # Mantém todas as linhas acima da linha chave (exclui a linha chave e todas as seguintes)
        df = df.iloc[:indice_linha_chave]
    
    # Extraindo apenas as linhas com todas as letras em maiúsculo
    letra = df['Causa - CID-BR-10'].str.isupper()
    df = df[letra]
    
    # Extrai informações do nome da aba
    partes = nome_aba.split('_')
    if len(partes) == 3:
        tipo, sexo, ano = partes[0], partes[1], partes[2]
    else:
        # Caso o nome da aba não siga o padrão esperado
        tipo, sexo, ano = 'Desconhecido', 'Desconhecido', 'Desconhecido'
    
    # Adiciona as colunas
    df['tipologia'] = tipo
    df['sexo'] = sexo
    df['ano'] = ano
    
    return df

def processar_todas_abas(caminho_arquivo):
    # Lista os nomes das abas
    nomes_abas = listar_abas(caminho_arquivo)
    
    # Lista para armazenar os DataFrames processados
    dataframes = []
    
    for nome_aba in nomes_abas:
        # Processa cada aba e adiciona à lista
        df_processado = processar_dataframe(caminho_arquivo, nome_aba)
        dataframes.append(df_processado)
        
    # Concatena todos os DataFrames em um único DataFrame
    df_concatenado = pd.concat(dataframes, ignore_index=True)
    
    return df_concatenado


--------------------------------------------------------------------------------------------------------------------------

### Estado: ALAGOAS 

In [3]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/AL_trienio.xlsx'

# Processa todas as abas
AL = processar_todas_abas(caminho_arquivo)
AL['estado'] = 'Alagoas'
AL

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.666667,0,0,0,0,0,0,1.333333,0.666667,...,0,0,0.333333,0.333333,2.333333,7.666667,IA,F,2018,Alagoas
1,032-052 NEOPLASIAS,0,0,0,0.333333,0,0,0.333333,0.333333,0.333333,...,3.333333,2,1.666667,1.666667,3.333333,18.333333,IA,F,2018,Alagoas
2,053-054 D SANGUE E ORG HEMAT E ALGUNS TRANS IM...,0,0,0,0,0,0,0,0.333333,0,...,0.333333,0,0.333333,0,0.333333,1.333333,IA,F,2018,Alagoas
3,"055-057 D ENDÓCRINAS, NUTRICIONAIS E METABÓLICAS",0.333333,0.333333,0,0,0,0,0,0,0.666667,...,2,1,1.666667,2.666667,3.666667,14,IA,F,2018,Alagoas
4,058-059 TRANSTORNOS MENTAIS E COMPORTAMENTAIS,0,0,0,0,0,0,0,0,0,...,0,0,0.333333,0.666667,0.333333,1.666667,IA,F,2018,Alagoas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,085-087 DOENÇAS DO APARELHO GENITURINÁRIO,0,0,0.333333,0.333333,0,0.333333,0,0,1,...,2,2,4,6,11.666667,34,U,M,2021,Alagoas
196,093-097 ALG AFECÇÕES ORIGIN NO PERÍODO PERINATAL,30.333333,0,0,0,0,0,0,0,0,...,0,0,0,0,0,30.333333,U,M,2021,Alagoas
197,"098-100 MALF CONGÊN, DEFORM E ANOMAL CROMOSSÔM...",6.333333,1,0,0.333333,0,0,0,0,0,...,0.333333,0,0,0,0.666667,9,U,M,2021,Alagoas
198,"101-103 SINT, SIN E ACH ANORM CLÍN E LAB, NCOP",0.666667,0.333333,0,0.666667,0.666667,0.666667,1,5.333333,6.666667,...,12.666667,16,15.333333,19.666667,40.666667,166.666667,U,M,2021,Alagoas


--------------------------------------------------------------------------------------------------------------------------

### Estado: Bahia 

In [4]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/BA_trienio.xlsx'
# Processa todas as abas
BA = processar_todas_abas(caminho_arquivo)
BA['estado'] = 'Bahia'
BA.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,4.333333,1,0.666667,0.333333,1,1,1,2,4.333333,...,5,11.666667,10,16.666667,45.666667,124,IA,F,2018,Bahia


--------------------------------------------------------------------------------------------------------------------------

### Estado: Ceará 

In [5]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/CE_trienio.xlsx'
# Processa todas as abas
CE = processar_todas_abas(caminho_arquivo)
CE['estado'] = 'Ceara'
CE.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,1.333333,0,0,0.666667,0,0.666667,0.666667,2.333333,1.666667,...,3.333333,6.333333,9.666667,9.333333,35.666667,83.666667,IA,F,2018,Ceara


--------------------------------------------------------------------------------------------------------------------------

### Estado: Espirito Santos 

In [6]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/ES_trienio.xlsx'
# Processa todas as abas
ES = processar_todas_abas(caminho_arquivo)
ES['estado'] = 'Espirito Santos'
ES.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0,0,0.333333,0,0,0,0.333333,0,0,...,0,0,0.333333,0.333333,1.333333,4.333333,IA,F,2018,Espirito Santos


--------------------------------------------------------------------------------------------------------------------------

### Estado: Maranhão

In [7]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/MA_trienio.xlsx'
# Processa todas as abas
MA = processar_todas_abas(caminho_arquivo)
MA['estado'] = 'Maranhao'
MA.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,3.333333,0.333333,0,0,0,0.666667,1.333333,1,0.666667,...,2.666667,1.666667,2,2,6.666667,25,RA,F,2018,Maranhao


--------------------------------------------------------------------------------------------------------------------------

### Estado: Minas Gerais 

In [8]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/MG_trienio.xlsx'
# Processa todas as abas
MG = processar_todas_abas(caminho_arquivo)
MG['estado'] = 'Minas Gerais'
MG.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,1.666667,2.333333,0,0.333333,1.333333,0.666667,1,0.666667,1.333333,...,6.333333,5.666667,7,6.666667,30.333333,77.666667,IA,F,2018,Minas Gerais


--------------------------------------------------------------------------------------------------------------------------

### Estado: Paraíba

In [9]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/PB_trienio.xlsx'
# Processa todas as abas
PB = processar_todas_abas(caminho_arquivo)
PB['estado'] = 'Paraiba'
PB.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,2,0,0.333333,0.666667,0.666667,0.333333,0,1.333333,0.333333,...,1,5.333333,5,6,26,57.666667,IA,F,2018,Paraiba


--------------------------------------------------------------------------------------------------------------------------

### Estado: Pernambuco

In [10]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/PE_trienio.xlsx'
# Processa todas as abas
PE = processar_todas_abas(caminho_arquivo)
PE['estado'] = 'Pernambuco'
PE.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,5.333333,1,0.666667,0.333333,1.666667,1.333333,2.666667,2.333333,3.666667,...,6,7.333333,13,12.333333,40.333333,114.333333,IA,F,2018,Pernambuco


--------------------------------------------------------------------------------------------------------------------------

### Estado: Piaui

In [11]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/PI_trienio.xlsx'
# Processa todas as abas
PI = processar_todas_abas(caminho_arquivo)
PI['estado'] = 'Piaui'
PI.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,1,0.666667,0,0,0.333333,0,1.666667,1,0.333333,...,2,1.666667,0.666667,3.666667,8.666667,29.333333,IA,F,2018,Piaui


--------------------------------------------------------------------------------------------------------------------------

### Estado: Rio Grande do Norte

In [12]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/RN_trienio.xlsx'
# Processa todas as abas
RN = processar_todas_abas(caminho_arquivo)
RN['estado'] = 'Rio Grande do Norte'
RN.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,1.333333,0.666667,0.666667,0,0.333333,0.666667,0.666667,1,1.666667,...,2.333333,3,3.333333,7.666667,20.666667,50.333333,IA,F,2018,Rio Grande do Norte


--------------------------------------------------------------------------------------------------------------------------

### Estado: Sergipe

In [13]:
# Especifica o caminho do arquivo Excel
caminho_arquivo = '../3_corrigindo os dados/1_metodologia alternativa_BASE_FINAL/SE_trienio.xlsx'
# Processa todas as abas
SE = processar_todas_abas(caminho_arquivo)
SE['estado'] = 'Sergipe'
SE.head(1)


,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.333333,0.666667,0,0,0,0.333333,0.333333,0.333333,0.333333,...,1,1.666667,3,1.333333,6.666667,20.333333,IA,F,2018,Sergipe


----

# BASE FINAL PROCESSADA 

In [14]:
# Lista com todos os DataFrames
dataframes = [AL,BA,CE,ES,MA,MG,PB,PE,PI,RN,SE]

# Concatena todos os DataFrames em um único DataFrame
base_final = pd.concat(dataframes, ignore_index=True)

# Exibe as primeiras linhas do DataFrame concatenado
base_final

,Causa - CID-BR-10,<1,1 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 anos e mais,Total,tipologia,sexo,ano,estado
0,001-031 ALGUMAS DOENÇAS INFECCIOSAS E PARASITÁ...,0.666667,0,0,0,0,0,0,1.333333,0.666667,...,0,0,0.333333,0.333333,2.333333,7.666667,IA,F,2018,Alagoas
1,032-052 NEOPLASIAS,0,0,0,0.333333,0,0,0.333333,0.333333,0.333333,...,3.333333,2,1.666667,1.666667,3.333333,18.333333,IA,F,2018,Alagoas
2,053-054 D SANGUE E ORG HEMAT E ALGUNS TRANS IM...,0,0,0,0,0,0,0,0.333333,0,...,0.333333,0,0.333333,0,0.333333,1.333333,IA,F,2018,Alagoas
3,"055-057 D ENDÓCRINAS, NUTRICIONAIS E METABÓLICAS",0.333333,0.333333,0,0,0,0,0,0,0.666667,...,2,1,1.666667,2.666667,3.666667,14,IA,F,2018,Alagoas
4,058-059 TRANSTORNOS MENTAIS E COMPORTAMENTAIS,0,0,0,0,0,0,0,0,0,...,0,0,0.333333,0.666667,0.333333,1.666667,IA,F,2018,Alagoas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,085-087 DOENÇAS DO APARELHO GENITURINÁRIO,0,0,0,0,0,0,0,0.666667,0,...,0.666667,2.333333,2.666667,2.333333,10.333333,20.333333,U,M,2021,Sergipe
2563,093-097 ALG AFECÇÕES ORIGIN NO PERÍODO PERINATAL,10.666667,0,0,0,0,0,0,0,0,...,0,0,0,0,0,10.666667,U,M,2021,Sergipe
2564,"098-100 MALF CONGÊN, DEFORM E ANOMAL CROMOSSÔM...",2.666667,1,0.666667,0.333333,0,0,0,0,0,...,0.333333,0.333333,0,0,0,5.333333,U,M,2021,Sergipe
2565,"101-103 SINT, SIN E ACH ANORM CLÍN E LAB, NCOP",0.333333,0.333333,0,0,1,0,1.333333,1,2.666667,...,5,6,5,8.666667,24.333333,72,U,M,2021,Sergipe


In [15]:
# Salva o DataFrame em um arquivo CSV
base_final.to_csv('base_mortalidade_antesdacorrecao.csv', index=False)